In [75]:
# import packages
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.express as px
import numpy as np

### Read Bins lists input 

In [44]:
# read the info about all bins
all_bins = pd.read_excel('input/FINAL_all_bins_all_clusters.xlsx')

In [58]:
all_bins = all_bins[['recovered_genome', 'sample', 'secondary_cluster_x', 'part',
       'completeness', 'contamination', 'strain_heterogeneity', 'length',
       'N50', 'Bin_qual', 'bst_genome_1st_layer', 'genome', 'final_genomes']]
all_bins.head(2)

,recovered_genome,sample,secondary_cluster_x,part,completeness,contamination,strain_heterogeneity,length,N50,Bin_qual,bst_genome_1st_layer,genome,final_genomes
0,V00H9L_vamb.216248.fa,V00H9L,p6-487_1,p6,96.55,1.72,0.0,2439293,206313,HQ,V00H9L_vamb.216248.fa,V00H9L_vamb.216248.fa,V00H9L_vamb.216248.fa
1,VEZUDR_vamb.198244.fa,VEZUDR,p6-487_1,p6,96.55,3.45,0.0,1903915,143532,HQ,V00H9L_vamb.216248.fa,V00H9L_vamb.216248.fa,V00H9L_vamb.216248.fa


In [54]:
best1 = pd.read_excel('input/FINAL_only_the_best_genomes_with_2nd_cluster_with_taxa_with_binsNmb_m.xlsx')
best1 = best1[['best_cluster_genome', 'fastani_ani', 'closest_placement_reference',
       'closest_placement_ani', 'domain', 'phylum', 'class', 'order', 'family',
       'genus', 'sp', 'GTDBv214_taxa_notes', 'number_of_bins_per_HQcluster']]
best1.tail(2)

,best_cluster_genome,fastani_ani,closest_placement_reference,closest_placement_ani,domain,phylum,class,order,family,genus,sp,GTDBv214_taxa_notes,number_of_bins_per_HQcluster
2261,VE8VBP_maxbin.031.fa,NaN,NaN,NaN,Bacteria,Bacillota_A,Clostridia,Christensenellales,UBA1242,UBA7868,NaN,unknown_sp,NaN
2262,VC0JXP_31410.fa,NaN,NaN,NaN,Bacteria,Bacillota_A,Clostridia,Oscillospirales,CAG-272,UMGS911,NaN,unknown_sp,NaN


In [59]:
hq_clust = all_bins.merge(best1, left_on='recovered_genome', right_on='best_cluster_genome', how='left')
hq_clust = hq_clust.drop(['best_cluster_genome'], axis=1)
hq_clust.tail(2)

,recovered_genome,sample,secondary_cluster_x,part,completeness,contamination,strain_heterogeneity,length,N50,Bin_qual,...,closest_placement_ani,domain,phylum,class,order,family,genus,sp,GTDBv214_taxa_notes,number_of_bins_per_HQcluster
70494,VZOC83_vamb.17294.fa,VZOC83,p1-320_2,p1,98.28,0.0,0.0,2542883,128883,HQ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70495,VZOC83_vamb.989.fa,VZOC83,p1-21_1,p1,98.28,0.0,0.0,1925701,67064,HQ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Missing bins for LQ clusterings

In [9]:
# read the list of missing bins (from first clustering)
missing_bins = pd.read_csv('input/missing_bins_list.csv')
missing_bins.tail(2)

,recovered_genome
15458,VZQLWU_vamb.295405.fa
15459,VZQLWU_vamb.332950.fa


In [6]:
# read the list of missing bins (from first clustering)
missing_bins = pd.read_csv('input/missing_bins_list.csv')

print('Number of bins in the first clustering:', len(all_bins))
print('Number of missing bins:', len(missing_bins))
print('Total number of bins:', len(all_bins) + len(missing_bins))

Number of bins in the first clustering: 70483
Number of missing bins: 15460
Total number of bins: 85943


### Merge list of missing genomes with clutering results

In [15]:
cdb = pd.read_csv('input/Cdb.csv')
cdb = cdb[['genome', 'primary_cluster','original_secondary_cluster', 'secondary_cluster']]
cdb['part'] = 'LQ_clusterings'
cdb['primary_cluster'] = 'LQ-'+cdb['primary_cluster'].astype(str)
cdb['secondary_cluster'] = 'LQ-'+cdb['secondary_cluster'].astype(str)
# read info about completeness and contamination
gen = pd.read_csv('input/genomeInformation.csv')
mrg = pd.merge(gen, cdb, on='genome', how='inner')
mrg.tail(2)


,genome,completeness,contamination,strain_heterogeneity,length,N50,centrality,primary_cluster,original_secondary_cluster,secondary_cluster,part
9472,VZSPJW_maxbin.036.fa,93.97,47.08,26.23,3167676,2880,0.963917,LQ-326,326_1,LQ-326_1,LQ_clusterings
9473,VZTMOW_vamb.164616.fa,67.63,0.00,0.00,1364300,30903,0.950937,LQ-738,738_2,LQ-738_1.2.3.4.5.9,LQ_clusterings


In [18]:
# Write stats
print('Number of bins passed filters to LQ clustering:', len(mrg), 'bins')
prc = str(len(mrg)/len(missing_bins)*100)[0:4]
print(prc,'%')
print('Nubmer of clusters in the LQ  clustering:', len(mrg['secondary_cluster'].unique()), 'clusters')

Number of bins passed filters to LQ clustering: 9474 bins
61.2 %
Nubmer of clusters in the LQ  clustering: 1644 clusters


In [92]:
miss_all_ClInfo = missing_bins.merge(mrg, left_on='recovered_genome', right_on='genome', how='left')
miss_all_ClInfo = miss_all_ClInfo.drop(['genome'], axis=1)
miss_all_ClInfo.tail(2)


,recovered_genome,sample,completeness,contamination,strain_heterogeneity,length,N50,centrality,primary_cluster,original_secondary_cluster,secondary_cluster,part,user_genome
15458,VZQLWU_vamb.295405.fa,VZQLWU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15459,VZQLWU_vamb.332950.fa,VZQLWU,68.97,1.9,100.0,3415643.0,66170.0,0.958679,LQ-531,531_1,LQ-531_1.3,LQ_clusterings,VZQLWU_vamb.332950


In [98]:
print('Bins with LQ cluster results:', len(miss_all_ClInfo[miss_all_ClInfo['secondary_cluster'].notnull()]))

Bins with LQ cluster results: 9474


### Add GTDB results to LQ clustering results

In [19]:
ar = pd.read_csv('input/gtdbtk.ar53.summary.tsv', sep='\t')
ar = ar[['user_genome','classification','fastani_ani','closest_placement_reference','closest_placement_ani']]

ar['domain'] = ar['classification'].str.split(';', expand=True)[0].str.replace(r'd__', '')
ar['phylum'] = ar['classification'].str.split(';', expand=True)[1].str.replace(r'p__', '')
ar['class'] = ar['classification'].str.split(';', expand=True)[2].str.replace(r'c__', '')
ar['order'] = ar['classification'].str.split(';', expand=True)[3].str.replace(r'o__', '')
ar['family'] = ar['classification'].str.split(';', expand=True)[4].str.replace(r'f__', '')
ar['genus'] = ar['classification'].str.split(';', expand=True)[5].str.replace(r'g__', '')
ar['sp'] = ar['classification'].str.split(';', expand=True)[6].str.replace(r's__', '')
ar = ar.drop('classification', axis=1)

ar.tail()

,user_genome,fastani_ani,closest_placement_reference,closest_placement_ani,domain,phylum,class,order,family,genus,sp
0,V61GT1_maxbin.144_sub,NaN,NaN,NaN,Archaea,Methanobacteriota,Methanobacteria,Methanobacteriales,Methanobacteriaceae,Methanobrevibacter_A,
1,VE7C2B_maxbin.163_sub,99.40,GCF_000016525.1,99.40,Archaea,Methanobacteriota,Methanobacteria,Methanobacteriales,Methanobacteriaceae,Methanobrevibacter_A,Methanobrevibacter_A smithii
2,VMYG00_maxbin.064_sub,97.82,GCF_023617305.1,97.82,Archaea,Methanobacteriota,Methanobacteria,Methanobacteriales,Methanobacteriaceae,Methanobrevibacter_A,Methanobrevibacter_A smithii_A
3,VOMLT7_maxbin.029_sub,NaN,NaN,NaN,Archaea,Methanobacteriota,Methanobacteria,Methanobacteriales,Methanobacteriaceae,Methanobrevibacter_A,


In [21]:
# read taxonimy annotation for the best genomes from cluster
gt = pd.read_csv('input/gtdbtk.bac120.summary.tsv', sep='\t')
gt = gt[['user_genome','classification','fastani_ani','closest_placement_reference','closest_placement_ani']]

gt['domain'] = gt['classification'].str.split(';', expand=True)[0].str.replace(r'd__', '')
gt['phylum'] = gt['classification'].str.split(';', expand=True)[1].str.replace(r'p__', '')
gt['class'] = gt['classification'].str.split(';', expand=True)[2].str.replace(r'c__', '')
gt['order'] = gt['classification'].str.split(';', expand=True)[3].str.replace(r'o__', '')
gt['family'] = gt['classification'].str.split(';', expand=True)[4].str.replace(r'f__', '')
gt['genus'] = gt['classification'].str.split(';', expand=True)[5].str.replace(r'g__', '')
gt['sp'] = gt['classification'].str.split(';', expand=True)[6].str.replace(r's__', '')
gt = gt.drop('classification', axis=1)

gt.tail(2)


,user_genome,fastani_ani,closest_placement_reference,closest_placement_ani,domain,phylum,class,order,family,genus,sp
1638,VZXIHP_vamb.46239,97.54,GCA_900543815.1,97.54,Bacteria,Bacteroidota,Bacteroidia,Bacteroidales,Bacteroidaceae,UBA4372,UBA4372 sp900543815
1639,VZZW6V_vamb.255038,97.15,GCA_900753975.1,97.15,Bacteria,Bacillota_A,Clostridia,TANB77,UBA1234,HGM13618,HGM13618 sp900753975


In [105]:
# Merge bacteria & archaea results
miss_best_taxa = pd.concat([gt, ar], ignore_index=True)
d = miss_all_ClInfo[['recovered_genome', 'secondary_cluster']].dropna()
d['recovered_genome'] = d['recovered_genome'].str.replace(r'.fa', '')
miss_best_taxa = miss_best_taxa.merge(d, left_on='user_genome', right_on='recovered_genome', how='left')
miss_best_taxa.drop('recovered_genome', axis=1, inplace=True)
miss_best_taxa.rename(columns={'user_genome':'final_genomes'}, inplace=True)
miss_best_taxa.tail(2)

/var/folders/5j/pvj6k02n3mn9ztpn77vjtfzw0000gn/T/ipykernel_34678/1999937258.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  d['recovered_genome'] = d['recovered_genome'].str.replace(r'.fa', '')


,user_genome,fastani_ani,closest_placement_reference,closest_placement_ani,domain,phylum,class,order,family,genus,sp,recovered_genome,secondary_cluster
1642,VMYG00_maxbin.064_sub,97.82,GCF_023617305.1,97.82,Archaea,Methanobacteriota,Methanobacteria,Methanobacteriales,Methanobacteriaceae,Methanobrevibacter_A,Methanobrevibacter_A smithii_A,VMYG00_maxbin.064_sub,LQ-105_1
1643,VOMLT7_maxbin.029_sub,NaN,NaN,NaN,Archaea,Methanobacteriota,Methanobacteria,Methanobacteriales,Methanobacteriaceae,Methanobrevibacter_A,,VOMLT7_maxbin.029_sub,LQ-110_1


### Add info about taxa to all clustering missing genomes

In [111]:
# Combine GTDB and LQ clustering results
lq_clust = pd.merge(miss_all_ClInfo, miss_best_taxa, on='secondary_cluster', how='left')
lq_clust = lq_clust.drop(['centrality','user_genome','primary_cluster', 'original_secondary_cluster'], axis=1)

lq_clust.tail(2)

,recovered_genome,sample,completeness,contamination,strain_heterogeneity,length,N50,centrality,primary_cluster,original_secondary_cluster,...,fastani_ani,closest_placement_reference,closest_placement_ani,domain,phylum,class,order,family,genus,sp
15458,VZQLWU_vamb.295405.fa,VZQLWU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15459,VZQLWU_vamb.332950.fa,VZQLWU,68.97,1.9,100.0,3415643.0,66170.0,0.958679,LQ-531,531_1,...,98.95,GCF_002222615.2,98.95,Bacteria,Bacteroidota,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,Bacteroides caccae


In [122]:
lq_clust.tail(2)

,recovered_genome,sample,completeness,contamination,strain_heterogeneity,length,N50,secondary_cluster,part,final_genomes,fastani_ani,closest_placement_reference,closest_placement_ani,domain,phylum,class,order,family,genus,sp
15458,VZQLWU_vamb.295405.fa,VZQLWU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15459,VZQLWU_vamb.332950.fa,VZQLWU,68.97,1.9,100.0,3415643.0,66170.0,LQ-531_1.3,LQ_clusterings,VUR0YN_vamb.182603,98.95,GCF_002222615.2,98.95,Bacteria,Bacteroidota,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,Bacteroides caccae


In [123]:
# write the final table
lq_clust.to_excel('results/FINAL_all_LQ_clusters.xlsx', index=False)
#lq_clust = pd.read_excel('results/FINAL_LQ_clusters.xlsx')
#lq_clust.tail(2)    

In [124]:
lq_clust['domain'].value_counts()

Bacteria    9467
Archaea        7
Name: domain, dtype: int64

In [126]:
archaea = lq_clust[lq_clust['domain'] == 'Archaea']
archaea.to_excel('results/FINAL_archaea_LQ_clusters.xlsx', index=False)

## Combine HQ_clusters and LQ clusters 

In [132]:
# lq clusters
lq_clust['Bin_qual'] = np.where(lq_clust['contamination'] > 10, 'LQ', np.where(lq_clust['contamination'].isna(), np.nan, 'MQ'))
value_counts = lq_clust['secondary_cluster'].value_counts()
value_counts = value_counts.reset_index().rename(columns={'index': 'value', 'secondary_cluster': 'count'})
lq_clust['number_of_bins_per_LQcluster'] = np.where(lq_clust['secondary_cluster'].isin(value_counts['value']), lq_clust['secondary_cluster'].map(value_counts.set_index('value')['count']), np.nan)

# hq clusters
hq_clust = hq_clust.drop(['genome'], axis=1)
hq_clust = hq_clust.rename(columns={'secondary_cluster_x':'secondary_cluster'})


In [151]:
#lq_clust.to_excel('results/FINAL_all_LQ_clusters.xlsx', index=False)
lq_clust = pd.read_excel('results/FINAL_all_LQ_clusters_m.xlsx')
lq_clust.tail(2)

,recovered_genome,sample,completeness,contamination,strain_heterogeneity,length,N50,secondary_cluster,part,final_genomes,...,domain,phylum,class,order,family,genus,sp,Bin_qual,number_of_bins_per_LQcluster,GTDBv214_taxa_notes
15458,VZQLWU_maxbin.027.fa,VZQLWU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15459,VZQLWU_vamb.295405.fa,VZQLWU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [153]:
lq_clust['GTDBv214_taxa_notes'].value_counts()

LQ_unknown_sp        580
LQ_unknown_genus      32
LQ_unknown_family     15
Name: GTDBv214_taxa_notes, dtype: int64

In [154]:
# Columns present in HQ clustering but not in LQ clustering
# Need to add to LQ clustering!
set(hq_clust.columns) - set(lq_clust.columns)


{'bst_genome_1st_layer', 'number_of_bins_per_HQcluster'}

In [155]:
# Columns present in LQ clustering but not in HQ clustering
# Need to remove from HQ clustering!
set(lq_clust.columns) - set(hq_clust.columns)

{'number_of_bins_per_LQcluster'}

In [156]:
all_clust = pd.concat([hq_clust, lq_clust], ignore_index=True)

In [157]:
all_clust.tail(2)

,recovered_genome,sample,secondary_cluster,part,completeness,contamination,strain_heterogeneity,length,N50,Bin_qual,...,domain,phylum,class,order,family,genus,sp,GTDBv214_taxa_notes,number_of_bins_per_HQcluster,number_of_bins_per_LQcluster
85954,VZQLWU_maxbin.027.fa,VZQLWU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85955,VZQLWU_vamb.295405.fa,VZQLWU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [160]:
all_clust.to_excel('results/FINAL_all_clusters.xlsx', index=False)